<a href="https://colab.research.google.com/github/DNason1999/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-06-10 01:24:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.04s   

2020-06-10 01:24:04 (552 KB/s) - ‘student.zip’ saved [20478/20478]



In [4]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [0]:
import pandas as pd

In [6]:
df = pd.read_csv('student-mat.csv', sep=';')
print(df.shape)
df.head()

(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [7]:
df.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [0]:
ind1 = ['sex', 'age']
ind2 = ['age', 'health']

In [0]:
dep1 = ['failures','G1']
dep2 = ['age', 'failures']

In [0]:
from scipy import stats

In [0]:
columns = ['chi2', 'p_value', 'dof', 'expected']
index = ['d1','d2','i1','i2']

In [0]:
data_dep1 = pd.Series(data=stats.chi2_contingency(pd.crosstab(df[dep1[0]], df[dep1[1]])), index=columns)
data_dep2 = pd.Series(data=stats.chi2_contingency(pd.crosstab(df[dep2[0]], df[dep2[1]])), index=columns)
data_ind1 = pd.Series(data=stats.chi2_contingency(pd.crosstab(df[ind1[0]], df[ind1[1]])), index=columns)
data_ind2 = pd.Series(data=stats.chi2_contingency(pd.crosstab(df[ind2[0]], df[ind2[1]])), index=columns)

In [13]:
results = pd.DataFrame([data_dep1,data_dep2,data_ind1,data_ind2], columns=columns, index=index)
results['p_value'] = results['p_value'].apply(lambda x : f"{x:.3f}")
results.head()

,chi2,p_value,dof,expected
d1,129.344587,0.000,48,"[[0.789873417721519, 0.789873417721519, 5.5291..."
d2,144.299039,0.000,21,"[[64.76962025316456, 10.379746835443038, 3.529..."
i1,5.994603,0.540,7,"[[43.17974683544304, 54.76455696202532, 51.605..."
i2,28.864970,0.419,28,"[[9.756962025316456, 9.341772151898734, 18.891..."


### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

Manual Chi^2 on dependent set 1

In [77]:
obvs = pd.crosstab(df[dep1[0]], df[dep1[1]])
obvs

age,15,16,17,18,19,20,21,22
sex,,,,,,,,
F,38,54,58,43,14,1,0,0
M,44,50,40,39,10,2,1,1


In [78]:
vals = obvs.values
vals

array([[38, 54, 58, 43, 14,  1,  0,  0],
       [44, 50, 40, 39, 10,  2,  1,  1]])

In [0]:
col_totals = df[dep1[0]].value_counts().sort_index().values
row_totals = df[dep1[1]].value_counts().sort_index().values

In [0]:
sample_size = df.shape[0]

In [0]:
a = col_totals[0] * row_totals / sample_size
b = col_totals[1] * row_totals / sample_size

In [101]:
expected = [list(a),list(b)]
expected

[[43.17974683544304,
  54.76455696202532,
  51.60506329113924,
  43.17974683544304,
  12.637974683544304,
  1.579746835443038,
  0.5265822784810127,
  0.5265822784810127],
 [38.82025316455696,
  49.23544303797468,
  46.39493670886076,
  38.82025316455696,
  11.362025316455696,
  1.420253164556962,
  0.47341772151898737,
  0.47341772151898737]]

In [102]:
vals

array([[38, 54, 58, 43, 14,  1,  0,  0],
       [44, 50, 40, 39, 10,  2,  1,  1]])

In [105]:
manual_chi_dep1 = ((vals - expected)**2 / expected).sum().sum()
manual_chi_dep1

5.99460281380294

In [109]:
data_dep1['chi2']

5.99460281380294

In [108]:
data_dep1['chi2'] == manual_chi_dep1

True

In [110]:
data_dep1['expected']==expected

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [0]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

---

- What is a sample "estimate" in statistics?

A sample estimate is an estimation of an entire population based on only a small sample of the population


---

- What are hypothesis test? How are they useful?

A Hypothesis test is when you create a null hypothesis stating something and an alternative hypothesis stating the opposite of the null. This can then be tested using various testing methods to determine if a hypothesis can be assumed to be true. Hypothesis tests are useful to determine if an observation can be reasonably assumed to be true based on a sample of data.

---

- What is a "null hypothesis?"

A null hypothesis is the assumption of the data which a statistician is trying to deem statistically plausible.


---

- What is a p-value and what does it represent?

p-value is the probability that the observation of a hypothesis test is due to random chance. A high p-value means that the null hypothesis cannot be rejected and is therefore plausible. A small p-value means that we reject the null hypothesis and assume the alternative hypothesis. 


---


- What does it mean for something to be "statistically significant?"

In statistical hypothesis testing, a result has statistical significance when it is very unlikely to have occurred given the null hypothesis.


---


- What is a test statistic and how does it relate to a p-value?

t-statistic is the ratio of the departure of the estimated value of a parameter from its hypothesized value to its standard error. In layman's terms: t-stat is the difference between the sample mean and the population mean divided by the standard error. The higher the t-stat, the smaller the p-value.


---


- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

In statistics, the number of degrees of freedom is the number of values in the final calculation of a statistic that are free to vary.
Degrees of freedom for:

* 1-sample: # of catergories -1
* 2-sample for independence: (# of Rows - 1) * (# of Columns - 1)

---




![One Sample T Statistic](https://www.statisticshowto.com/wp-content/uploads/2015/09/t-score.jpg)

x¯  = passed in iterable

μ = assumed mean of the population 

because the sample standard error (denominator of whole fraction) contains the n term, as the sample size increases, the sample standard error will decrease, therefore the sample t-stastic will become more representative of the true population t-statistic as the sample size increases

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)